In [1]:
import chess
import chess.pgn
import matplotlib.pyplot as plt
import numpy as np
from IPython import display
import time
from sklearn import preprocessing

In [2]:
path = 'lichess_NotGmSule_2023-02-01.pgn'

In [3]:
file = open(path, encoding="utf-8")
games = []
a = chess.pgn.read_game(file)
while a:
    games.append(a)
    try:
        a = chess.pgn.read_game(file)
    except:
        print(f'could not read game number {len(games)}')

In [4]:
def make_move(uci):
    '''
    returns the two postions of the uci(Universal Chess Interface) format
    '''
    pos1 = uci[:2]
    pos2 = uci[2:4]
    return (chess.parse_square(pos1), chess.parse_square(pos2))


In [5]:
def get_moves(game):
    moves = []
    for i in game.mainline_moves():
        moves.append(i)
    return moves

In [6]:
import numpy as np
import chess

def get_piece_type(piece):
    # Helper function to get the type of a chess piece (with color information)
    if piece is None:
        return None
    elif piece.color == chess.WHITE:
        if piece.piece_type == chess.PAWN:
            return 'P'
        elif piece.piece_type == chess.KNIGHT:
            return 'N'
        elif piece.piece_type == chess.BISHOP:
            return 'B'
        elif piece.piece_type == chess.ROOK:
            return 'R'
        elif piece.piece_type == chess.QUEEN:
            return 'Q'
        elif piece.piece_type == chess.KING:
            return 'K'
    elif piece.color == chess.BLACK:
        if piece.piece_type == chess.PAWN:
            return 'p'
        elif piece.piece_type == chess.KNIGHT:
            return 'n'
        elif piece.piece_type == chess.BISHOP:
            return 'b'
        elif piece.piece_type == chess.ROOK:
            return 'r'
        elif piece.piece_type == chess.QUEEN:
            return 'q'
        elif piece.piece_type == chess.KING:
            return 'k'

def get_board_matrix(board_state):
    # Initialize an 8x8 matrix to represent the board
    matrix = np.zeros((8, 8), dtype=str)

    # Parse the board_state string to obtain the positions of the pieces on the board
    board = chess.Board(board_state)
    for row in range(8):
        for col in range(8):
            # Get the square index corresponding to the current row and column
            square = chess.square(col, 7 - row)

            # Get the type of the piece occupying the current square (if any)
            piece_type = get_piece_type(board.piece_at(square))

            # Store the piece type in the matrix
            matrix[row][col] = piece_type or '.'

    return matrix


In [7]:
get_board_matrix(games[0].board().fen())

array([['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'],
       ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
       ['.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.'],
       ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
       ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']], dtype='<U1')

In [8]:
le = preprocessing.LabelEncoder()
le.fit_transform(get_board_matrix(games[0].board().fen()).flatten())

array([12,  9,  7, 11,  8,  7,  9, 12, 10, 10, 10, 10, 10, 10, 10, 10,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  4,  4,
        4,  4,  4,  4,  4,  6,  3,  1,  5,  2,  1,  3,  6], dtype=int64)

In [9]:
# iterate through all the moves in a game and label encode them

def label_encode_game(game):
    board = game.board()
    moves = get_moves(game)
    encoded_moves = []
    for move in moves:
        board.push(move)
        encoded_moves.append(le.fit_transform(get_board_matrix(board.fen()).flatten()))
    return encoded_moves

In [10]:
label_encode_game(games[0])

[array([12,  9,  7, 11,  8,  7,  9, 12, 10, 10, 10, 10, 10, 10, 10, 10,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  4,  4,
         4,  0,  4,  4,  4,  6,  3,  1,  5,  2,  1,  3,  6], dtype=int64),
 array([12,  9,  7, 11,  8,  7,  9, 12, 10, 10,  0, 10, 10, 10, 10, 10,  0,
         0, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  4,  4,
         4,  0,  4,  4,  4,  6,  3,  1,  5,  2,  1,  3,  6], dtype=int64),
 array([12,  9,  7, 11,  8,  7,  9, 12, 10, 10,  0, 10, 10, 10, 10, 10,  0,
         0, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  4,  4,
         4,  0,  0,  4,  4,  6,  3,  1,  5,  2,  1,  3,  6], dtype=int64),
 array([12,  9,  7, 11,  8,  7,  9, 12, 10, 10,  0,  0, 10, 10, 10, 10,  0,
         0, 10,

In [12]:
# make a neural network that takes in a board state and outputs the next move using tensorflow

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import models
from keras import optimizers
from keras import losses
from keras import metrics
from keras import regularizers
from keras import callbacks

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(64,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(64, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
                loss=losses.categorical_crossentropy,
                metrics=[metrics.categorical_accuracy])

model.summary()

# train the model on the first 100 games

for game in games[:100]:
    encoded_game = label_encode_game(game)
    for i in range(len(encoded_game)-1):
        x = encoded_game[i]
        y = encoded_game[i+1] - encoded_game[i]
        x = x.reshape(1, 64)
        y = y.reshape(1, 64)
        model.fit(x, y, epochs=1, batch_size=1)

# test the model on the first 100 games


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 64)                4160      
                                                                 
 dense_22 (Dense)            (None, 64)                4160      
                                                                 
 dense_23 (Dense)            (None, 64)                4160      
                                                                 
 dense_24 (Dense)            (None, 64)                4160      
                                                                 
 dense_25 (Dense)            (None, 64)                4160      
                                                                 
 dense_26 (Dense)            (None, 64)                4160      
                                                                 
 dense_27 (Dense)            (None, 64)               

In [15]:
# test the model by giving it a board state and seeing if it can predict the next move

game = game[0]

encoded_game = label_encode_game(game)
for i in range(len(encoded_game)-1):
    x = encoded_game[i]
    y = encoded_game[i+1] - encoded_game[i]
    x = x.reshape(1, 64)
    y = y.reshape(1, 64)
    print(model.predict(x))



1/1 [==============================] - 0s 195ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
1/1 [==============================] - 0s 24ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
1/1 [==============================] - 0s 30ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
1/1 [==============================] - 0s 25ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [ ]:
# train on random forest

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)

for game in games[:100]:
    encoded_game = label_encode_game(game)
    for i in range(len(encoded_game)-1):
        x = encoded_game[i]
        y = encoded_game[i+1] - encoded_game[i]
        x = x.reshape(1, 64)
        y = y.reshape(1, 64)
        rf.fit(x, y)